<a href="https://colab.research.google.com/github/RafinEazdan/Handwritten_Digit_Recognition/blob/main/MNISTwithUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio tensorflow

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# Reshape for TensorFlow (batch, height, width, channels)
features_train = X_train.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
features_test = X_test.reshape((-1, 28, 28, 1)).astype("float32") / 255.0

# Convert labels to one-hot encoding
targets_train = to_categorical(y_train, 10)
targets_test = to_categorical(y_test, 10)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(28, 28, 1)),
    Activation('relu'),
    BatchNormalization(),

    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3)),
    Activation('relu'),
    BatchNormalization(),

    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    BatchNormalization(),
    Dense(512),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(10, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=7,
    width_shift_range=0.05,
    height_shift_range=0.07,
    shear_range=0.2,
    zoom_range=0.05
)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(features_train, targets_train, batch_size=64)
test_generator = test_datagen.flow(features_test, targets_test, batch_size=64)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(features_train) // 64,
    epochs=5,
    validation_data=test_generator,
    validation_steps=len(features_test) // 64
)

# Optional: Evaluate final performance
score = model.evaluate(features_test, targets_test)
print(f'Test accuracy: {score[1]:.2f}')

# Save model
model.save("mnist_cnn_model.h5")

# Load model (for prediction use)
model = tf.keras.models.load_model("mnist_cnn_model.h5")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_train original shape (60000, 28, 28)
y_train original shape (60000,)
X_test original shape (10000, 28, 28)
y_test original shape (10000,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


937/937 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - accuracy: 0.9242 - loss: 0.2528 - val_accuracy: 0.9881 - val_loss: 0.0404
Epoch 2/5
  1/937 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9688 - loss: 0.0752

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step - accuracy: 0.9688 - loss: 0.0752 - val_accuracy: 0.9882 - val_loss: 0.0408
Epoch 3/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9824 - loss: 0.0589 - val_accuracy: 0.9901 - val_loss: 0.0317
Epoch 4/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 1.0000 - loss: 0.0255 - val_accuracy: 0.9898 - val_loss: 0.0320
Epoch 5/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - accuracy: 0.9861 - loss: 0.0454 - val_accuracy: 0.9918 - val_loss: 0.0256
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9903 - loss: 0.0305


Test accuracy: 0.99


In [7]:
from PIL import Image
import gradio as gr
import numpy as np


def predict_digit(img: Image.Image):
    img = img.convert("L").resize((28, 28))
    img = np.array(img).astype("float32") / 255.0

    # Invert image if background is white
    if np.mean(img) > 0.5:
        img = 1 - img

    img = img.reshape(1, 28, 28, 1)
    prediction = model.predict(img)[0]
    return {str(i): float(prediction[i]) for i in range(10)}

In [ ]:
# Gradio app
interface = gr.Interface(
    fn=predict_digit,
    inputs = gr.Image(type="pil", label="Upload an Image of a Digit"),
    outputs=gr.Label(num_top_classes=3),
    title="MNIST Digit Classifier",
    description="Upload a digit image (white background, black number ideally) to classify it using a CNN trained on MNIST."
    )

interface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eff0fc768feb27626f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 728ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
